In [63]:
import arcpy
from arcpy import env, mp
import pandas as pd
import requests
import os
import zipfile
import numpy as np

landcover_url = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'
out_dir = r'C:\Users\Eli Schlossberg\Documents\ArcGIS\Projects\MyProject'
  
output_crs = arcpy.SpatialReference(26915)
output_landcover = os.path.join(out_dir, 'reprojected_nlcd.tif')

In [64]:
def download_landcover(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Downloaded: {save_path}")
    else:
        print(f"Failed to download: {url}")
    with zipfile.ZipFile(save_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.join(out_dir, 'landcover'))

def crs_qa_step(dem, output_dem):
    desc = arcpy.Describe(dem)
    current_crs = desc.spatialReference

    # Check if the CRS is the desired one
    if current_crs != output_crs:
        print(f"Reprojecting {dem} from {current_crs.name} to {output_crs.name}")
        # Reproject the feature class
        arcpy.Project_management(dem, output_dem, output_crs)
    else:
        print(f"{fc} is already in the desired CRS ({output_crs.name})")

In [66]:
# Download the .tif file
download_landcover(dem_url, os.path.join(out_dir, 'landcover.zip'))
landcover_path = os.path.join(out_dir, r'landcover', r'NLCD_2019_Land_Cover.tif')
landcover = arcpy.Raster(landcover_path)

Downloaded: C:\Users\Eli Schlossberg\Documents\ArcGIS\Projects\MyProject\landcover.zip


RuntimeError: ERROR 000732: Input Raster: Dataset C:\Users\Eli Schlossberg\Documents\ArcGIS\Projects\MyProject\landcover\NLCD_2019_Land_Cover.tif does not exist or is not supported

In [40]:
crs_qa_step(landcover_path, output_landcover)

Reprojecting C:\Users\Eli Schlossberg\Documents\ArcGIS\Projects\MyProject\dem\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m from NAD_1983_UTM_Zone_15N to NAD_1983_UTM_Zone_15N


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Dataset or Feature Class: Dataset C:\Users\Eli Schlossberg\Documents\ArcGIS\Projects\MyProject\dem\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m does not exist or is not supported
Failed to execute (Project).
